In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, glob
import matplotlib
import numpy as np
from geopandas.tools import geocode
import time
import os, json, urllib, requests, webbrowser

In [2]:
df_desc = pd.read_excel('./data/datasets_description.xlsx')
df_desc.head()

,Database,Description,Output
0,Aquifer_Auser,Information about the Auser aquifer. This wate...,"Depth_to_Groundwater_SAL, Depth_to_Groundwater..."
1,Water_Spring_Amiata,Information about the Amiata aquifer. This aqu...,"Flow_Rate_Bugnano, Flow_Rate_Arbure,\n Flow_Ra..."
2,Aquifer_Petrignano,Information about Petrignano aquifer. \nIt is ...,"Depth_to_Groundwater_P24,\n Depth_to_Groundwat..."
3,Aquifer_Doganella,Information about Doganella aquifer. The Dogan...,"Depth_to_Groundwater_Pozzo_1, Depth_to_Groundw..."
4,Aquifer_Luco,Information about Luco aquifer. It is an under...,Depth_to_Groundwater_Podere_Casetta


In [3]:
## collect all geolocations of header names
# manual check changes
manual_checks = {'Monte Serra': 'Centro Televisivo Monte Serra', 'Piaggione': 'Piaggione, Lucca', 'Monteporzio': 'Monte Porzio Catone', 
'Monticiano la Pineta': 'Pinete, Monticiano', 'Monteroni Arbia Biena': "Monteroni d'Arbia", 'Ponte Orgia': 'Orgia', 'Petrignano': 'Petrignano, Perugia',
'S Piero': 'San Piero a Sieve', 'Le Croci': 'Le Croci, Barberino di Mugello', 'S Agata': "Sant'Agata, Firenze", 'Consuma': 'Passo della Consuma, Arezzo',
'S Savino': 'Monte San Savino, Arezzo', 'S Fiora': 'Santa Fiora, Grosseto', 'Terni': '05100 Terni'} # Laghetto Verde still wrong location

if os.path.exists('./data/geolocations.pckl'):
    gdf = pd.read_pickle('./data/geolocations.pckl')
else:
    gdf = gpd.GeoDataFrame()
    for file in glob.glob('./data/*.csv'):
            df = pd.read_csv(file)

            feat = file.split('_')[-1].replace('.csv', '')
            cols = [c for c in set(df.columns) if 'Depth' not in c and 'Flow_Rate' not in c and 'Volume' not in c and 'Lake_Level' not in c and 'Hydrometry' not in c]
            cols = set(cols) - set(['Date'])
            params = [c.split('_')[0] for c in cols]
            places = [c.replace('Rainfall', '').replace('Temperature', '').replace('_', ' ').strip() for c in cols]
            places_edit = list(map(manual_checks.get, places, places)) # replace the manual (google) checks

            locs = ["{}, Italy".format(name) for name in places_edit]
            
            # set up geocoder
            from geopy.geocoders import Nominatim
            geolocator = Nominatim(user_agent="acea-water-test")

            from geopy.extra.rate_limiter import RateLimiter
            geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

            gdfs = gpd.GeoDataFrame({'query': list(locs), 'place': list(places), 'place_edit': list(places_edit), 'param': list(params)})
            gdfs = gdfs.groupby('place').agg({'param': lambda x: 'Both' if len(list(x)) > 1 else x, 'query': 'first'}) # agg. locations with both temperature & rainfall measurements
            gdfs = gdfs.reset_index()

            gdfs['location'] = gdfs['query'].apply(geocode)
            gdfs['adress'] = gdfs['location'].apply(lambda loc: loc.address if loc else None)
            gdfs['feat'] = feat

            from shapely.geometry import Point
            gdfs = gpd.GeoDataFrame(gdfs, geometry=gdfs['location'].apply(lambda loc: Point(loc.longitude, loc.latitude) if loc else None), crs='EPSG:4326')
            gdf = pd.concat([gdf, gdfs], axis=0)
            
    gdf.to_pickle('./data/geolocations.pckl')
    gdf.head()

In [12]:
# for the map
gdf['color'] = pd.factorize(gdf['feat'])[0]
gdf['lat'] = gdf['geometry'].y
gdf['lon'] = gdf['geometry'].x

import folium
icons = {'Rainfall': 'cloud', 'Temperature': 'sun-o', 'Both': 'flag'} # rainfall cloud, temperature sun, both (temperature & rainfall) flag
colors = ['navy', 'lightblue', 'blue', 'darkblue', 'darkgreen', 'orange', 'pink', 'purple', 'darkpurple'] # blues aquifers, green lake, orange river, pinks water springs

m = folium.Map(location=[43.15, 12.07], zoom_start=8, tiles='cartodbpositron')
for i, row in gdf.iterrows():
    folium.Marker([row['lat'], row['lon']], tooltip='<b>name:</b> {}<br><b>parameters:</b> {}<br><b>adress:</b> {}<br><b>waterbody:</b> {}'.format(row['place'], row['param'], row['adress'], row['feat']), icon=folium.Icon(color=colors[row['color']], icon=icons.get(row['param']), prefix='fa')).add_to(m)

m

In [5]:
# collect some relevant metrics from nasa POWER

output = "JSON"
locations = [gdfs.geometry.values]

output_folder = r'./data/nasa-power'
params = 'QV2M,RH2M,PS,PRECTOT,T2M_RANGE,T2MDEW,T2MWET,WS10M_RANGE,WS50M_RANGE'
base_url = "https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=SinglePoint&tempAverage=DAILY&parameters={}&startDate=20000101&endDate=20210101&lat={latitude}&lon={longitude}&outputList={output}&userCommunity=SSE"
gdfs['longitude'] = gdfs.geometry.x
gdfs['latitude'] = gdfs.geometry.y

for i, row in gdfs.iloc[:].iterrows():
    longitude = row.longitude
    latitude = row.latitude
    api_request_url = base_url.format(params, longitude=longitude, latitude=latitude, output=output.upper())

    json_response = json.loads(requests.get(api_request_url).content.decode('utf-8'))

    # Selects the file URL from the JSON response
    csv_request_url = json_response['outputs'][output.lower()]

    # Download File to Folder
    output_file_location = os.path.join(output_folder, row.place)+'.json'
    urllib.request.urlretrieve(csv_request_url, output_file_location)

In [32]:
for f in glob.glob('./data/nasa-power/*.json'):
    with open(f) as file:
        data = json.load(file)
    df = pd.DataFrame(data['features'][0]['properties']['parameter'])
    df.index = pd.to_datetime(df.index)
    df.to_pickle(f.replace('.json', '.pckl'))